In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [7]:
train_lre_30sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [8]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [9]:
y_30sec_labels = train_lre_30sec["lang"]

In [10]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [11]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [12]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [13]:
y_30sec_labels=le.transform(y_30sec_labels)

In [14]:
y_30sec_labels.shape

(114276,)

In [15]:
nb_classes = 14
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [16]:
Y_train_labels.shape

(114276, 14)

In [17]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [18]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [19]:
#Shuffle the Dataset
X_train_10sec_F,  X_train_30sec_F, Y_train_labels = shuffle(X_train_10sec, X_train_30sec, Y_train_labels, random_state=0)

In [20]:
import torch.nn as nn

In [21]:
class FC_Network(torch.nn.Module):

    def __init__(self,):
        super(FC_Network, self).__init__()

        D_in = D_out = 500        # param['patch_length'] * param['n_channels'] * (param['n_fft'] / 2 + 1)
        num_nodes_fnn = 2048
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_4 = torch.nn.Linear(num_nodes_fnn,num_nodes_fnn)
        self.bn_4 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        out = x
        #out = x.view(x.size(0), -1)
        out = self.bn_1(self.relu(self.layer_1(out)))
        out = self.bn_2(self.relu(self.layer_2(out)))
        out = self.bn_3(self.relu(self.layer_3(out)))
        out = self.bn_4(self.relu(self.layer_4(out)))
        out = self.output_layer(out)

        #out = (self.relu(self.layer_1(out)))
        #out = (self.relu(self.layer_2(out)))
        #out = (self.relu(self.layer_3(out)))
        #out = (self.relu(self.layer_4(out)))
        #out = (self.output_layer(out))
        #out = out.view(x.size())
        return out

In [22]:
G = FC_Network()

In [23]:
G.cuda()

FC_Network(
  (layer_1): Linear(in_features=500, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (relu): ReLU()
)

In [24]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [25]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
#D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [26]:
len(X_train_10sec)

114276

In [27]:
len(X_train_30sec)

114276

In [28]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train_10sec_F) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train_10sec_F[start:end]

        c = X_train_30sec_F[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))
        X = X.cuda()
        c = c.cuda()
        # Dicriminator forward-loss-backward-update
        G_sample = G(X)
        #D_real = D(X, c)
        #D_fake = D(G_sample, c)

        #D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        #D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        #D_loss = D_loss_real + D_loss_fake

        #D_loss.backward()
        #D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        #D_fake = D(G_sample, c)
        G_loss = criterion(G_sample, c)
        #G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()
        
        G_solver.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        #print('Iter-{}; G_loss: {}'.format(it, G_loss.data.numpy()))



        it+= batch_size
    print('G_loss: {}'.format(G_loss.cpu().data.numpy()))
           

epoch: 0
G_loss: [ 0.94095123]
epoch: 1
G_loss: [ 0.87068075]
epoch: 2
G_loss: [ 0.83763647]
epoch: 3
G_loss: [ 0.81275499]
epoch: 4
G_loss: [ 0.79290026]
epoch: 5
G_loss: [ 0.77705646]
epoch: 6
G_loss: [ 0.7637648]
epoch: 7
G_loss: [ 0.75219822]
epoch: 8
G_loss: [ 0.74163282]
epoch: 9
G_loss: [ 0.73206562]
epoch: 10
G_loss: [ 0.72336411]
epoch: 11
G_loss: [ 0.71529996]
epoch: 12
G_loss: [ 0.70782906]
epoch: 13
G_loss: [ 0.70012438]
epoch: 14
G_loss: [ 0.69327748]
epoch: 15
G_loss: [ 0.68667084]
epoch: 16
G_loss: [ 0.68053752]
epoch: 17
G_loss: [ 0.67481411]
epoch: 18
G_loss: [ 0.66917163]
epoch: 19
G_loss: [ 0.66373843]
epoch: 20
G_loss: [ 0.65861213]
epoch: 21
G_loss: [ 0.65358317]
epoch: 22
G_loss: [ 0.64911181]
epoch: 23
G_loss: [ 0.64409661]
epoch: 24
G_loss: [ 0.63974047]
epoch: 25
G_loss: [ 0.63497823]
epoch: 26
G_loss: [ 0.63042814]
epoch: 27
G_loss: [ 0.6262995]
epoch: 28
G_loss: [ 0.62200755]
epoch: 29
G_loss: [ 0.61797601]
epoch: 30
G_loss: [ 0.61393821]
epoch: 31
G_loss: [ 

In [29]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec_F))

In [30]:
G.eval()

FC_Network(
  (layer_1): Linear(in_features=500, out_features=2048)
  (bn_1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=2048, out_features=2048)
  (bn_2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=2048, out_features=2048)
  (bn_3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=2048, out_features=2048)
  (bn_4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=2048, out_features=500)
  (relu): ReLU()
)

In [34]:
gen_samples = G(X_train_30sec_gen.cuda())

In [35]:
gen_samples.data.shape

torch.Size([114276, 500])

In [36]:
#Taking the generated iVectors we will try to check the acc by MLP

In [37]:
train_X_gen = gen_samples.cpu().data.numpy()

In [38]:
type(train_X_gen)

numpy.ndarray

In [39]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [40]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_05Jan_10sec.csv')


In [41]:
val_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,uttid,language_code,data_source,speech_duration,segmentid1
0,-0.195398,-0.390386,-0.624269,1.535433,0.388117,1.727448,0.463109,2.059767,0.165206,0.619942,...,0.616712,-2.654504,0.482690,0.401316,-1.639292,lre17_zpuysjrb,zho-nan,mls14,10,lre17_zpuysjrb
1,-0.629280,-0.063858,0.397267,-0.205915,0.102730,2.412635,1.007062,1.123458,2.397956,1.009312,...,-0.626052,1.390082,0.393644,1.559527,-1.124127,lre17_ljfobgxa,eng-usg,mls14,10,lre17_ljfobgxa
2,0.896452,0.746032,-0.547790,1.955862,-0.198707,1.430271,0.535072,1.199097,2.253311,0.050932,...,-1.859781,-0.889882,-1.823634,-1.507139,-0.811382,lre17_guhfzbxl,zho-cmn,mls14,10,lre17_guhfzbxl
3,0.323422,0.577406,1.585811,-0.071801,-0.398092,1.788636,1.529049,1.042687,-0.557043,1.131491,...,0.519515,1.404486,-0.052511,0.367571,1.274410,lre17_tbywroez,eng-usg,mls14,10,lre17_tbywroez
4,1.384084,0.814175,1.261965,1.078853,-1.601087,0.756707,1.157398,0.146018,-0.566307,-0.446297,...,-0.850699,0.355410,-1.455804,2.629600,-0.044268,lre17_phsyjibn,spa-lac,mls14,10,lre17_phsyjibn


In [42]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [43]:
X_val = X_val.values
#X_eval = X_eval.values

In [44]:
#X_eval.shape

In [45]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [46]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [47]:
y_val_labels = le.transform(y_val)

In [48]:
y_val_labels[0:25]

array([13,  5, 12,  5, 11, 11, 11, 12, 10,  0, 11,  0,  2, 13, 12,  6, 13,
        4,  6,  0, 11,  2,  4,  3, 10])

In [49]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
#Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [50]:
Y_val

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [51]:
X_val = X_val.astype('float32')


In [52]:
X_val_10sec_gen = Variable(torch.from_numpy(X_val))

In [53]:
gen_samples_val = G(X_val_10sec_gen.cuda())

In [54]:
gen_samples_val.data.shape

torch.Size([928, 500])

In [55]:
X_val_gen = gen_samples_val.cpu().data.numpy()

In [63]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 14)                7182      
__________

In [65]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [66]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [67]:
batch_size = 256
nb_epoch = 15

In [70]:
#train_X_gen
#Checking Accuracy with augmented data 
history = model.fit(train_X_gen, Y_train_labels , batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val_gen , Y_val),callbacks=[saveBestModel])

Train on 114276 samples, validate on 928 samples
Epoch 1/15
Epoch 00000: val_acc did not improve
8s - loss: 0.0193 - acc: 0.9943 - val_loss: 1.6437 - val_acc: 0.7058
Epoch 2/15
Epoch 00001: val_acc did not improve
9s - loss: 0.0164 - acc: 0.9950 - val_loss: 1.8181 - val_acc: 0.6994
Epoch 3/15
Epoch 00002: val_acc did not improve
8s - loss: 0.0158 - acc: 0.9951 - val_loss: 1.8349 - val_acc: 0.7080
Epoch 4/15
Epoch 00003: val_acc did not improve
8s - loss: 0.0157 - acc: 0.9955 - val_loss: 1.8358 - val_acc: 0.7004
Epoch 5/15
Epoch 00004: val_acc did not improve
8s - loss: 0.0172 - acc: 0.9950 - val_loss: 1.7838 - val_acc: 0.7058
Epoch 6/15
Epoch 00005: val_acc did not improve
8s - loss: 0.0173 - acc: 0.9947 - val_loss: 1.9974 - val_acc: 0.6972
Epoch 7/15
Epoch 00006: val_acc did not improve
8s - loss: 0.0174 - acc: 0.9948 - val_loss: 1.8307 - val_acc: 0.7112
Epoch 8/15
Epoch 00007: val_acc did not improve
8s - loss: 0.0167 - acc: 0.9955 - val_loss: 1.9153 - val_acc: 0.7026
Epoch 9/15
Epoc

In [71]:
score = model.evaluate(X_val_gen, Y_val, verbose=0)
print('Acc %:', score[1])

Acc %: 0.703663793103


In [77]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


In [ ]:
classif = SVC(C=1, kernel='rbf', gamma='auto', coef0=1, shrinking=True, random_state=0,
                      probability=False, tol=1e-3, cache_size=1e4, class_weight='balanced')
classif.fit(X_train_10sec_F, y_30sec_labels)

In [ ]:
classif.classes_

In [ ]:
acc = metrics.accuracy_score(y_val_labels, classif.predict(X_val))

In [ ]:
acc